In [86]:
#!pip install catboost

In [1]:
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

from catboost import CatBoostRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
print(f"catBoost version: {cb.__version__}")
print(f"numpy version: {np.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"seaborn version: {sns.__version__}")

catBoost version: 1.2.2
numpy version: 1.26.0
pandas version: 2.1.1
seaborn version: 0.12.2


In [3]:
print('Loading Properties ...')
properties2016 = pd.read_csv('properties_2016.csv', low_memory = False)
properties2017 = pd.read_csv('properties_2017.csv', low_memory = False)

print('Loading Train ...')
train2016 = pd.read_csv('train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
train2017 = pd.read_csv('train_2017.csv', parse_dates=['transactiondate'], low_memory=False)

print('Loading Sample ...')
sample_submission = pd.read_csv('sample_submission.csv', low_memory = False)

print('Done ...')

Loading Properties ...
Loading Train ...
Loading Sample ...
Done ...


In [4]:
print(properties2016.shape)
print(properties2017.shape)
print(train2016.shape)
print(train2017.shape)
print(sample_submission.shape)

(2985217, 58)
(2985217, 58)
(90275, 3)
(77613, 3)
(2985217, 7)


## Exploring the csvs

In [5]:
train2016.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,1/1/2016
1,14366692,-0.1684,1/1/2016
2,12098116,-0.0040,1/1/2016
3,12643413,0.0218,2/1/2016
4,14432541,-0.0050,2/1/2016


In [6]:
train2017.head()

,parcelid,logerror,transactiondate
0,14297519,0.025595,1/1/2017
1,17052889,0.055619,1/1/2017
2,14186244,0.005383,1/1/2017
3,12177905,-0.103410,1/1/2017
4,10887214,0.006940,1/1/2017


In [7]:
properties2016.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [8]:
properties2017.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [9]:
sample_submission.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


## Observing sample submission parcelID order

In [10]:
sample_submission["ParcelId"].equals(properties2016["parcelid"])

True

In [11]:
sample_submission["ParcelId"].equals(properties2017["parcelid"])

False

The parcelid order in sample submission seems to follow that of 2016, thus we need to reorder 2017

### Reordering properties2017

In [12]:
properties2017 = pd.merge(sample_submission["ParcelId"], properties2017, how='left', left_on ='ParcelId',right_on='parcelid').drop("ParcelId", axis=1)
properties2017

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,168176230,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,110245.0,226900.0,2016.0,116655.0,3035.26,NaN,NaN,NaN
2985213,14273630,NaN,NaN,NaN,3.0,3.0,NaN,NaN,3.0,NaN,...,NaN,NaN,378466.0,762157.0,2016.0,383691.0,8653.10,NaN,NaN,NaN
2985214,168040630,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,NaN,NaN,201575.0,483188.0,2016.0,281613.0,5671.40,NaN,NaN,NaN
2985215,168040830,NaN,NaN,NaN,3.0,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,347863.0,578621.0,2016.0,230758.0,5894.36,NaN,NaN,NaN


In [13]:
sample_submission["ParcelId"].equals(properties2017["parcelid"])

True

## Splitting transaction dates into month, year (removing day)

In [14]:
def split_date(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"], dayfirst=True)
    df["transaction_year"] = df["transactiondate"].dt.year
    
#     df["transaction_month"] = df["transactiondate"].dt.month

#     the code below is used if we want to let 2017 jan start as month 13 (might be useful when we combine both years to train a model)
    df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month

    
#     df["transaction_day"] = df["transactiondate"].dt.day
    
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

train2016 = split_date(train2016)
train2017 = split_date(train2017)

In [15]:
print(train2016.shape)
train2016.head()

(90275, 4)


,parcelid,logerror,transaction_year,transaction_month
0,11016594,0.0276,2016,1
1,14366692,-0.1684,2016,1
2,12098116,-0.0040,2016,1
3,12643413,0.0218,2016,1
4,14432541,-0.0050,2016,1


In [16]:
print(train2017.shape)
train2017.head()

(77613, 4)


,parcelid,logerror,transaction_year,transaction_month
0,14297519,0.025595,2017,13
1,17052889,0.055619,2017,13
2,14186244,0.005383,2017,13
3,12177905,-0.103410,2017,13
4,10887214,0.006940,2017,13


we can see that Jan 2017 starts as month 13

## Non-Null counts for the columns in properties.csv

In [17]:
properties2016.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   parcelid                      2985217 non-null  int64  
 1   airconditioningtypeid         811519 non-null   float64
 2   architecturalstyletypeid      6061 non-null     float64
 3   basementsqft                  1628 non-null     float64
 4   bathroomcnt                   2973755 non-null  float64
 5   bedroomcnt                    2973767 non-null  float64
 6   buildingclasstypeid           12629 non-null    float64
 7   buildingqualitytypeid         1938488 non-null  float64
 8   calculatedbathnbr             2856305 non-null  float64
 9   decktypeid                    17096 non-null    float64
 10  finishedfloor1squarefeet      202717 non-null   float64
 11  calculatedfinishedsquarefeet  2929652 non-null  float64
 12  finishedsquarefeet12        

## Possible cleaning of dataframe  

 ### 1) remove the features with 1 unique value 

In [18]:
oneUniqueValueList2016 = []

for i in properties2016.columns:
    uniqueNum = properties2016[i].nunique()
    
    if uniqueNum == 1:
        oneUniqueValueList2016.append(i)
    
    
oneUniqueValueList2016

['decktypeid',
 'hashottuborspa',
 'poolcnt',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'storytypeid',
 'fireplaceflag',
 'taxdelinquencyflag']

In [19]:
oneUniqueValueList2017 = []

for i in properties2017.columns:
    uniqueNum = properties2017[i].nunique()
    
    if uniqueNum == 1:
        oneUniqueValueList2017.append(i)
    
    
oneUniqueValueList2017

['decktypeid',
 'hashottuborspa',
 'poolcnt',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'storytypeid',
 'fireplaceflag',
 'taxdelinquencyflag']

Both 2016 and 2017 have the same columns that only has 1 unique value, thus we will be removing them

### Removing above elements from our df

In [20]:
properties2016.drop(oneUniqueValueList2016, inplace = True, axis=1)
properties2017.drop(oneUniqueValueList2017, inplace = True, axis=1)

print(properties2016.shape)
print(properties2017.shape)

(2985217, 49)
(2985217, 49)


## Labelling our numerical & categorical columns

In [21]:
nominal_features = ["censustractandblock",
                    "airconditioningtypeid",
                    "architecturalstyletypeid",
                    "buildingclasstypeid",
                    "fips",
                    "heatingorsystemtypeid",
                    "propertycountylandusecode",
                    "propertylandusetypeid",
                    "regionidcounty",
                    "typeconstructiontypeid",
                    "rawcensustractandblock",
                    "regionidneighborhood",
                    "propertyzoningdesc",
                    "regionidcity",
                    "regionidzip",                  
                   ]

ordinal_features = ["bathroomcnt",
                    "bedroomcnt",
                    "buildingqualitytypeid",
                    "calculatedbathnbr",
                    "threequarterbathnbr",
                    "fireplacecnt",
                    "fullbathcnt",
                    "garagecarcnt",
                    "numberofstories",
                    "roomcnt",
                    "unitcnt",
                    "taxdelinquencyyear",
                    "yearbuilt",
                    "assessmentyear"] 

categorical_features = nominal_features + ordinal_features

#we have to include the transaction date individual features that we created
categorical_features += ["transaction_year", "transaction_month"]

numeric_features = ["basementsqft",
                    "finishedfloor1squarefeet",
                    "calculatedfinishedsquarefeet",
                    "finishedsquarefeet6",
                    "finishedsquarefeet12",
                    "finishedsquarefeet13",
                    "finishedsquarefeet15",
                    "finishedsquarefeet50",
                    "garagetotalsqft",
                    "latitude",
                    "longitude",
                    "lotsizesquarefeet",
                    "poolsizesum",
                    "yardbuildingsqft17",
                    "yardbuildingsqft26",
                    "taxvaluedollarcnt",
                    "structuretaxvaluedollarcnt",
                    "landtaxvaluedollarcnt",
                    "taxamount"]

In [22]:
print(f"numeric features length: {len(numeric_features)}")
print(f"categorical features length: {len(categorical_features)}")

numeric features length: 19
categorical features length: 31


## Preparing our train and test data

- our first approach is to combine both 2016 and 2017 data and train a model that predicts both years together
- we have to merge train.csv with their respective properties.csv


In [23]:
# how = left is equivalent to a left join in SQL, thus we will only have a dataframe with houses that are sold
combined2016 = pd.merge(train2016, properties2016, how = 'left', on = 'parcelid')  
combined2017 = pd.merge(train2017, properties2017, how = 'left', on = 'parcelid')

print(combined2016.shape)
print(combined2017.shape)

(90275, 52)
(77613, 52)


In [24]:
#we can see the merging of both train and properties
combined2016.head()

,parcelid,logerror,transaction_year,transaction_month,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016,1,1.0,NaN,NaN,2.0,3.0,NaN,...,NaN,1959.0,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,6.037107e+13
1,14366692,-0.1684,2016,1,NaN,NaN,NaN,3.5,4.0,NaN,...,NaN,2014.0,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN
2,12098116,-0.0040,2016,1,1.0,NaN,NaN,3.0,2.0,NaN,...,NaN,1940.0,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,6.037464e+13
3,12643413,0.0218,2016,1,1.0,NaN,NaN,2.0,2.0,NaN,...,NaN,1987.0,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,6.037296e+13
4,14432541,-0.0050,2016,1,NaN,NaN,NaN,2.5,4.0,NaN,...,NaN,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,6.059042e+13


In [25]:
# combining 2016 and 2017 for the final df
combined = pd.concat([combined2016, combined2017])
combined

,parcelid,logerror,transaction_year,transaction_month,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
0,11016594,0.027600,2016,1,1.0,NaN,NaN,2.0,3.0,NaN,...,NaN,1959.0,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,6.037107e+13
1,14366692,-0.168400,2016,1,NaN,NaN,NaN,3.5,4.0,NaN,...,NaN,2014.0,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN
2,12098116,-0.004000,2016,1,1.0,NaN,NaN,3.0,2.0,NaN,...,NaN,1940.0,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,6.037464e+13
3,12643413,0.021800,2016,1,1.0,NaN,NaN,2.0,2.0,NaN,...,NaN,1987.0,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,6.037296e+13
4,14432541,-0.005000,2016,1,NaN,NaN,NaN,2.5,4.0,NaN,...,NaN,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017,21,1.0,NaN,NaN,3.0,3.0,NaN,...,NaN,1980.0,NaN,265000.0,379000.0,2016.0,114000.0,4685.34,NaN,6.037132e+13
77609,11000655,0.020615,2017,21,NaN,NaN,NaN,2.0,2.0,NaN,...,NaN,1940.0,NaN,70917.0,354621.0,2016.0,283704.0,4478.43,NaN,6.037101e+13
77610,17239384,0.013209,2017,21,NaN,NaN,NaN,2.0,4.0,NaN,...,NaN,1964.0,1.0,50683.0,67205.0,2016.0,16522.0,1107.48,NaN,6.111008e+13
77611,12773139,0.037129,2017,21,1.0,NaN,NaN,1.0,3.0,NaN,...,NaN,1954.0,NaN,32797.0,49546.0,2016.0,16749.0,876.43,NaN,6.037434e+13


## Populating NaN values (0 for numerical columns, empty string for categorical columns)

In [26]:
combined[numeric_features] = combined[numeric_features].fillna(0)

In [27]:
combined[categorical_features] = combined[categorical_features].fillna('')

## Converting all categorical feature values to strings (a requirement for catBoost)

In [28]:
for i in categorical_features:
    combined[i] = combined[i].astype(str)

In [29]:
combined

,parcelid,logerror,transaction_year,transaction_month,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
0,11016594,0.027600,2016,1,1.0,,0.0,2.0,3.0,,...,0.0,1959.0,,122754.0,360170.0,2015.0,237416.0,6735.88,,60371066461001.0
1,14366692,-0.168400,2016,1,,,0.0,3.5,4.0,,...,0.0,2014.0,,346458.0,585529.0,2015.0,239071.0,10153.02,,
2,12098116,-0.004000,2016,1,1.0,,0.0,3.0,2.0,,...,0.0,1940.0,,61994.0,119906.0,2015.0,57912.0,11484.48,,60374638003004.0
3,12643413,0.021800,2016,1,1.0,,0.0,2.0,2.0,,...,0.0,1987.0,,171518.0,244880.0,2015.0,73362.0,3048.74,,60372963002002.0
4,14432541,-0.005000,2016,1,,,0.0,2.5,4.0,,...,0.0,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,,60590423381006.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017,21,1.0,,0.0,3.0,3.0,,...,0.0,1980.0,,265000.0,379000.0,2016.0,114000.0,4685.34,,60371319001006.0
77609,11000655,0.020615,2017,21,,,0.0,2.0,2.0,,...,0.0,1940.0,,70917.0,354621.0,2016.0,283704.0,4478.43,,60371014003002.0
77610,17239384,0.013209,2017,21,,,0.0,2.0,4.0,,...,0.0,1964.0,1.0,50683.0,67205.0,2016.0,16522.0,1107.48,,61110084022016.0
77611,12773139,0.037129,2017,21,1.0,,0.0,1.0,3.0,,...,0.0,1954.0,,32797.0,49546.0,2016.0,16749.0,876.43,,60374338022005.0


## Getting our train / test splits


In [30]:
y = combined["logerror"]
X = combined.drop(["parcelid", "logerror"], axis=1)

In [31]:
print(X.shape)
X.head()

(167888, 50)


,transaction_year,transaction_month,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
0,2016,1,1.0,,0.0,2.0,3.0,,4.0,2.0,...,0.0,1959.0,,122754.0,360170.0,2015.0,237416.0,6735.88,,60371066461001.0
1,2016,1,,,0.0,3.5,4.0,,,3.5,...,0.0,2014.0,,346458.0,585529.0,2015.0,239071.0,10153.02,,
2,2016,1,1.0,,0.0,3.0,2.0,,4.0,3.0,...,0.0,1940.0,,61994.0,119906.0,2015.0,57912.0,11484.48,,60374638003004.0
3,2016,1,1.0,,0.0,2.0,2.0,,4.0,2.0,...,0.0,1987.0,,171518.0,244880.0,2015.0,73362.0,3048.74,,60372963002002.0
4,2016,1,,,0.0,2.5,4.0,,,2.5,...,0.0,1981.0,2.0,169574.0,434551.0,2015.0,264977.0,5488.96,,60590423381006.0


In [32]:
y.head()

0    0.0276
1   -0.1684
2   -0.0040
3    0.0218
4   -0.0050
Name: logerror, dtype: float64

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

print(f"X_train size: {len(X_train)}")
print(f"X_test size: {len(X_test)}")
print(f"y_train size: {len(y_train)}")
print(f"y_test size: {len(y_test)}")

X_train size: 134310
X_test size: 33578
y_train size: 134310
y_test size: 33578


In [34]:
X_train.head()

,transaction_year,transaction_month,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
1145,2016,1,,,0.0,3.0,2.0,,1.0,3.0,...,0.0,1981.0,,254938.0,410564.0,2015.0,155626.0,5008.74,,60372641031011.0
46858,2017,18,,,0.0,2.0,3.0,,6.0,2.0,...,0.0,1947.0,,35960.0,108540.0,2016.0,72580.0,1353.25,,60371411021004.0
26074,2017,16,,,0.0,2.5,3.0,,,2.5,...,0.0,1990.0,2.0,256156.0,512312.0,2016.0,256156.0,5692.00,,61110076141139.0
43052,2016,6,,,0.0,2.0,3.0,,4.0,2.0,...,0.0,1926.0,,97580.0,166558.0,2015.0,68978.0,2174.84,,60372141004000.0
76357,2016,9,,,0.0,1.0,1.0,,,1.0,...,0.0,1930.0,2.0,10547.0,410000.0,2015.0,399453.0,4320.64,,60590320111002.0


In [35]:
y_train.head()

1145     0.045000
46858   -2.271428
26074    0.008852
43052   -0.098700
76357    0.035400
Name: logerror, dtype: float64

## Before we train a catBoost model, we need to get indices of categorical columns from X_train

In [36]:
categorical_indices = []
for i in categorical_features:
    idx = X_train.columns.get_loc(i)
    categorical_indices.append(idx)

categorical_indices.sort()
       
print(categorical_indices)

[0, 1, 2, 3, 5, 6, 7, 8, 9, 17, 18, 19, 20, 22, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 45, 48, 49]


## Preparing a submission onto kaggle

In [37]:
datelist = list(sample_submission.columns[1:])
print(datelist)
sample_submission

['201610', '201611', '201612', '201710', '201711', '201712']


,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2985212,168176230,0,0,0,0,0,0
2985213,14273630,0,0,0,0,0,0
2985214,168040630,0,0,0,0,0,0
2985215,168040830,0,0,0,0,0,0


## Function to preprocess our data as we did above

In [38]:
def preprocess(df):
    df[numeric_features] = df[numeric_features].fillna(0)
    df[categorical_features] = df[categorical_features].fillna('')
    
    for i in categorical_features:
        df[i] = df[i].astype(str)

def makeTransactionDF(df, month, year):
    data = {'transaction_year': [year for i in range(len(df))],
            'transaction_month': [month for i in range(len(df))]}
    
    transaction_month_year = pd.DataFrame(data)
    test_month_year = pd.concat([transaction_month_year, df], axis=1)
    
    return test_month_year

---
# Using oct 2016 to predict for all 6 time points 

1) Making a transaction df for October 2016

In [39]:
test_oct_2016 = makeTransactionDF(properties2016, 10, 2016)

In [40]:
X_test_oct_2016 = test_oct_2016.drop('parcelid', axis=1)

print(X_test_oct_2016.shape)
X_test_oct_2016.head()

(2985217, 50)


,transaction_year,transaction_month,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
0,2016,10,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN
1,2016,10,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN
2,2016,10,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN
3,2016,10,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,...,NaN,1948.0,1.0,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN
4,2016,10,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,...,NaN,1947.0,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN


In [41]:
preprocess(X_test_oct_2016)

In [42]:
X_test_oct_2016

,transaction_year,transaction_month,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
0,2016,10,,,0.0,0.0,0.0,,,,...,0.0,,,0.0,9.0,2015.0,9.0,0.00,,
1,2016,10,,,0.0,0.0,0.0,,,,...,0.0,,,0.0,27516.0,2015.0,27516.0,0.00,,
2,2016,10,,,0.0,0.0,0.0,,,,...,0.0,,,650756.0,1413387.0,2015.0,762631.0,20800.37,,
3,2016,10,,,0.0,0.0,0.0,3.0,7.0,,...,0.0,1948.0,1.0,571346.0,1156834.0,2015.0,585488.0,14557.57,,
4,2016,10,,,0.0,0.0,0.0,4.0,,,...,0.0,1947.0,,193796.0,433491.0,2015.0,239695.0,5725.17,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,2016,10,,,0.0,,,,,,...,0.0,,,0.0,0.0,,0.0,0.00,,
2985213,2016,10,,,0.0,,,,,,...,0.0,,,0.0,0.0,,0.0,0.00,,
2985214,2016,10,,,0.0,,,,,,...,0.0,,,0.0,0.0,,0.0,0.00,,
2985215,2016,10,,,0.0,,,,,,...,0.0,,,0.0,0.0,,0.0,0.00,,


# Training and Prediction

## 1. With Ensemble (500 iterations)

In [65]:
num_ensembles = 5
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    model = CatBoostRegressor(
        iterations=500, 
        learning_rate=0.03,
        depth=6, 
        l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        cat_features = categorical_indices,
        random_seed=i)
    
    model.fit(X_train, y_train)

    y_pred += model.predict(X_test_oct_2016)
    
y_pred /= num_ensembles

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

0:	learn: 0.0688215	total: 1.66s	remaining: 13m 46s
1:	learn: 0.0688048	total: 2.96s	remaining: 12m 16s
2:	learn: 0.0687882	total: 4.02s	remaining: 11m 6s
3:	learn: 0.0687730	total: 5.32s	remaining: 10m 59s
4:	learn: 0.0687561	total: 6.64s	remaining: 10m 57s
5:	learn: 0.0687409	total: 7.82s	remaining: 10m 43s
6:	learn: 0.0687275	total: 9.08s	remaining: 10m 39s
7:	learn: 0.0687137	total: 10.4s	remaining: 10m 36s
8:	learn: 0.0687018	total: 11.5s	remaining: 10m 27s
9:	learn: 0.0686895	total: 12.8s	remaining: 10m 24s
10:	learn: 0.0686778	total: 13.9s	remaining: 10m 19s
11:	learn: 0.0686660	total: 15.1s	remaining: 10m 14s
12:	learn: 0.0686563	total: 16.5s	remaining: 10m 17s
13:	learn: 0.0686470	total: 17.7s	remaining: 10m 13s
14:	learn: 0.0686362	total: 18.9s	remaining: 10m 10s
15:	learn: 0.0686235	total: 20.1s	remaining: 10m 6s
16:	learn: 0.0686133	total: 21.2s	remaining: 10m 3s
17:	learn: 0.0686052	total: 22.3s	remaining: 9m 58s
18:	learn: 0.0685944	total: 23.5s	remaining: 9m 53s
19:	lear

156:	learn: 0.0680856	total: 3m 9s	remaining: 6m 54s
157:	learn: 0.0680834	total: 3m 11s	remaining: 6m 53s
158:	learn: 0.0680821	total: 3m 12s	remaining: 6m 52s
159:	learn: 0.0680801	total: 3m 13s	remaining: 6m 51s
160:	learn: 0.0680782	total: 3m 14s	remaining: 6m 49s
161:	learn: 0.0680759	total: 3m 15s	remaining: 6m 48s
162:	learn: 0.0680743	total: 3m 17s	remaining: 6m 47s
163:	learn: 0.0680719	total: 3m 18s	remaining: 6m 46s
164:	learn: 0.0680685	total: 3m 19s	remaining: 6m 44s
165:	learn: 0.0680672	total: 3m 20s	remaining: 6m 43s
166:	learn: 0.0680652	total: 3m 21s	remaining: 6m 42s
167:	learn: 0.0680641	total: 3m 22s	remaining: 6m 40s
168:	learn: 0.0680625	total: 3m 23s	remaining: 6m 39s
169:	learn: 0.0680610	total: 3m 25s	remaining: 6m 37s
170:	learn: 0.0680597	total: 3m 26s	remaining: 6m 36s
171:	learn: 0.0680584	total: 3m 27s	remaining: 6m 35s
172:	learn: 0.0680555	total: 3m 28s	remaining: 6m 34s
173:	learn: 0.0680547	total: 3m 29s	remaining: 6m 32s
174:	learn: 0.0680519	total: 

309:	learn: 0.0678972	total: 6m 10s	remaining: 3m 47s
310:	learn: 0.0678963	total: 6m 12s	remaining: 3m 46s
311:	learn: 0.0678950	total: 6m 13s	remaining: 3m 44s
312:	learn: 0.0678936	total: 6m 14s	remaining: 3m 43s
313:	learn: 0.0678925	total: 6m 15s	remaining: 3m 42s
314:	learn: 0.0678916	total: 6m 16s	remaining: 3m 41s
315:	learn: 0.0678900	total: 6m 17s	remaining: 3m 40s
316:	learn: 0.0678889	total: 6m 19s	remaining: 3m 38s
317:	learn: 0.0678880	total: 6m 20s	remaining: 3m 37s
318:	learn: 0.0678865	total: 6m 21s	remaining: 3m 36s
319:	learn: 0.0678855	total: 6m 22s	remaining: 3m 35s
320:	learn: 0.0678829	total: 6m 23s	remaining: 3m 33s
321:	learn: 0.0678817	total: 6m 24s	remaining: 3m 32s
322:	learn: 0.0678809	total: 6m 26s	remaining: 3m 31s
323:	learn: 0.0678801	total: 6m 27s	remaining: 3m 30s
324:	learn: 0.0678795	total: 6m 28s	remaining: 3m 29s
325:	learn: 0.0678781	total: 6m 29s	remaining: 3m 27s
326:	learn: 0.0678774	total: 6m 30s	remaining: 3m 26s
327:	learn: 0.0678766	total:

463:	learn: 0.0677193	total: 9m 10s	remaining: 42.7s
464:	learn: 0.0677179	total: 9m 11s	remaining: 41.5s
465:	learn: 0.0677168	total: 9m 12s	remaining: 40.3s
466:	learn: 0.0677156	total: 9m 14s	remaining: 39.2s
467:	learn: 0.0677143	total: 9m 15s	remaining: 38s
468:	learn: 0.0677134	total: 9m 16s	remaining: 36.8s
469:	learn: 0.0677122	total: 9m 17s	remaining: 35.6s
470:	learn: 0.0677106	total: 9m 18s	remaining: 34.4s
471:	learn: 0.0677098	total: 9m 19s	remaining: 33.2s
472:	learn: 0.0677078	total: 9m 21s	remaining: 32s
473:	learn: 0.0677067	total: 9m 22s	remaining: 30.8s
474:	learn: 0.0677051	total: 9m 23s	remaining: 29.7s
475:	learn: 0.0677039	total: 9m 24s	remaining: 28.5s
476:	learn: 0.0677028	total: 9m 25s	remaining: 27.3s
477:	learn: 0.0677023	total: 9m 27s	remaining: 26.1s
478:	learn: 0.0677010	total: 9m 28s	remaining: 24.9s
479:	learn: 0.0676999	total: 9m 29s	remaining: 23.7s
480:	learn: 0.0676980	total: 9m 30s	remaining: 22.5s
481:	learn: 0.0676968	total: 9m 31s	remaining: 21.

...

0:	learn: 0.0688216	total: 1.26s	remaining: 10m 28s
1:	learn: 0.0688041	total: 2.51s	remaining: 10m 24s
2:	learn: 0.0687874	total: 3.73s	remaining: 10m 18s
3:	learn: 0.0687717	total: 4.92s	remaining: 10m 10s
4:	learn: 0.0687585	total: 6.18s	remaining: 10m 11s
5:	learn: 0.0687451	total: 7.44s	remaining: 10m 12s
6:	learn: 0.0687323	total: 8.82s	remaining: 10m 20s
7:	learn: 0.0687210	total: 10.1s	remaining: 10m 20s
8:	learn: 0.0687075	total: 11.2s	remaining: 10m 8s
9:	learn: 0.0686965	total: 12.3s	remaining: 10m 1s
10:	learn: 0.0686833	total: 13.5s	remaining: 9m 59s
11:	learn: 0.0686728	total: 14.6s	remaining: 9m 55s
12:	learn: 0.0686618	total: 15.8s	remaining: 9m 52s
13:	learn: 0.0686517	total: 17.1s	remaining: 9m 52s
14:	learn: 0.0686411	total: 18.2s	remaining: 9m 49s
15:	learn: 0.0686305	total: 19.4s	remaining: 9m 47s
16:	learn: 0.0686199	total: 20.6s	remaining: 9m 45s
17:	learn: 0.0686114	total: 21.9s	remaining: 9m 45s
18:	learn: 0.0686027	total: 23.2s	remaining: 9m 46s
19:	learn: 0.0

156:	learn: 0.0681013	total: 3m 12s	remaining: 7m 1s
157:	learn: 0.0680998	total: 3m 14s	remaining: 7m
158:	learn: 0.0680979	total: 3m 15s	remaining: 6m 59s
159:	learn: 0.0680968	total: 3m 16s	remaining: 6m 58s
160:	learn: 0.0680943	total: 3m 17s	remaining: 6m 56s
161:	learn: 0.0680929	total: 3m 18s	remaining: 6m 54s
162:	learn: 0.0680910	total: 3m 20s	remaining: 6m 53s
163:	learn: 0.0680895	total: 3m 21s	remaining: 6m 52s
164:	learn: 0.0680870	total: 3m 22s	remaining: 6m 51s
165:	learn: 0.0680852	total: 3m 23s	remaining: 6m 49s
166:	learn: 0.0680829	total: 3m 24s	remaining: 6m 48s
167:	learn: 0.0680807	total: 3m 26s	remaining: 6m 47s
168:	learn: 0.0680792	total: 3m 27s	remaining: 6m 45s
169:	learn: 0.0680783	total: 3m 28s	remaining: 6m 44s
170:	learn: 0.0680770	total: 3m 29s	remaining: 6m 42s
171:	learn: 0.0680752	total: 3m 30s	remaining: 6m 41s
172:	learn: 0.0680731	total: 3m 31s	remaining: 6m 40s
173:	learn: 0.0680712	total: 3m 33s	remaining: 6m 39s
174:	learn: 0.0680701	total: 3m 3

310:	learn: 0.0679077	total: 6m 11s	remaining: 3m 46s
311:	learn: 0.0679058	total: 6m 12s	remaining: 3m 45s
312:	learn: 0.0679050	total: 6m 13s	remaining: 3m 43s
313:	learn: 0.0679040	total: 6m 14s	remaining: 3m 42s
314:	learn: 0.0679031	total: 6m 15s	remaining: 3m 41s
315:	learn: 0.0679019	total: 6m 16s	remaining: 3m 40s
316:	learn: 0.0679012	total: 6m 17s	remaining: 3m 38s
317:	learn: 0.0679000	total: 6m 18s	remaining: 3m 37s
318:	learn: 0.0678994	total: 6m 19s	remaining: 3m 36s
319:	learn: 0.0678986	total: 6m 20s	remaining: 3m 34s
320:	learn: 0.0678968	total: 6m 22s	remaining: 3m 33s
321:	learn: 0.0678961	total: 6m 23s	remaining: 3m 32s
322:	learn: 0.0678949	total: 6m 24s	remaining: 3m 31s
323:	learn: 0.0678936	total: 6m 25s	remaining: 3m 30s
324:	learn: 0.0678922	total: 6m 27s	remaining: 3m 29s
325:	learn: 0.0678914	total: 6m 28s	remaining: 3m 27s
326:	learn: 0.0678901	total: 6m 29s	remaining: 3m 26s
327:	learn: 0.0678890	total: 6m 30s	remaining: 3m 25s
328:	learn: 0.0678884	total:

464:	learn: 0.0677366	total: 9m 8s	remaining: 41.3s
465:	learn: 0.0677358	total: 9m 9s	remaining: 40.2s
466:	learn: 0.0677354	total: 9m 10s	remaining: 39s
467:	learn: 0.0677351	total: 9m 11s	remaining: 37.8s
468:	learn: 0.0677340	total: 9m 12s	remaining: 36.6s
469:	learn: 0.0677335	total: 9m 13s	remaining: 35.4s
470:	learn: 0.0677321	total: 9m 14s	remaining: 34.2s
471:	learn: 0.0677317	total: 9m 16s	remaining: 33.1s
472:	learn: 0.0677298	total: 9m 17s	remaining: 31.9s
473:	learn: 0.0677281	total: 9m 18s	remaining: 30.7s
474:	learn: 0.0677266	total: 9m 19s	remaining: 29.5s
475:	learn: 0.0677260	total: 9m 21s	remaining: 28.4s
476:	learn: 0.0677255	total: 9m 22s	remaining: 27.2s
477:	learn: 0.0677241	total: 9m 23s	remaining: 26s
478:	learn: 0.0677212	total: 9m 24s	remaining: 24.8s
479:	learn: 0.0677205	total: 9m 25s	remaining: 23.6s
480:	learn: 0.0677200	total: 9m 26s	remaining: 22.4s
481:	learn: 0.0677194	total: 9m 27s	remaining: 21.3s
482:	learn: 0.0677171	total: 9m 29s	remaining: 20.1s

100%|█████████████████████████████████████████████████████████████████████████████████| 5/5 [1:03:58<00:00, 767.64s/it]


## Populating our submission df

In [68]:
for i in datelist:
    sample_submission[i] = y_pred

## Downloading it to csv

In [70]:
sample_submission

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.023276,0.023276,0.023276,0.023276,0.023276,0.023276
1,10759547,0.025207,0.025207,0.025207,0.025207,0.025207,0.025207
2,10843547,0.031657,0.031657,0.031657,0.031657,0.031657,0.031657
3,10859147,0.027092,0.027092,0.027092,0.027092,0.027092,0.027092
4,10879947,0.016236,0.016236,0.016236,0.016236,0.016236,0.016236
...,...,...,...,...,...,...,...
2985212,168176230,0.008617,0.008617,0.008617,0.008617,0.008617,0.008617
2985213,14273630,0.008617,0.008617,0.008617,0.008617,0.008617,0.008617
2985214,168040630,0.008617,0.008617,0.008617,0.008617,0.008617,0.008617
2985215,168040830,0.008617,0.008617,0.008617,0.008617,0.008617,0.008617


In [71]:
sample_submission.to_csv("submission.csv", index=False)

## 2. Without Ensemble (500 iterations)

In [73]:
model = CatBoostRegressor(
    iterations=500, 
    learning_rate=0.03,
    depth=6, 
    l2_leaf_reg=3,
    loss_function='MAE',
    eval_metric='MAE',
    cat_features = categorical_indices
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test_oct_2016)

0:	learn: 0.0688215	total: 1.33s	remaining: 11m 3s
1:	learn: 0.0688048	total: 2.56s	remaining: 10m 38s
2:	learn: 0.0687882	total: 3.58s	remaining: 9m 53s
3:	learn: 0.0687730	total: 4.8s	remaining: 9m 55s
4:	learn: 0.0687561	total: 6.04s	remaining: 9m 58s
5:	learn: 0.0687409	total: 7.12s	remaining: 9m 46s
6:	learn: 0.0687275	total: 8.29s	remaining: 9m 43s
7:	learn: 0.0687137	total: 9.44s	remaining: 9m 40s
8:	learn: 0.0687018	total: 10.3s	remaining: 9m 19s
9:	learn: 0.0686895	total: 10.7s	remaining: 8m 43s
10:	learn: 0.0686778	total: 11.1s	remaining: 8m 12s
11:	learn: 0.0686660	total: 11.5s	remaining: 7m 49s
12:	learn: 0.0686563	total: 12s	remaining: 7m 30s
13:	learn: 0.0686470	total: 13.1s	remaining: 7m 34s
14:	learn: 0.0686362	total: 14.3s	remaining: 7m 40s
15:	learn: 0.0686235	total: 15.3s	remaining: 7m 43s
16:	learn: 0.0686133	total: 16.6s	remaining: 7m 51s
17:	learn: 0.0686052	total: 17.7s	remaining: 7m 54s
18:	learn: 0.0685944	total: 18.8s	remaining: 7m 55s
19:	learn: 0.0685860	tot

156:	learn: 0.0680856	total: 3m 5s	remaining: 6m 46s
157:	learn: 0.0680834	total: 3m 7s	remaining: 6m 45s
158:	learn: 0.0680821	total: 3m 8s	remaining: 6m 44s
159:	learn: 0.0680801	total: 3m 9s	remaining: 6m 43s
160:	learn: 0.0680782	total: 3m 11s	remaining: 6m 42s
161:	learn: 0.0680759	total: 3m 12s	remaining: 6m 41s
162:	learn: 0.0680743	total: 3m 13s	remaining: 6m 40s
163:	learn: 0.0680719	total: 3m 14s	remaining: 6m 39s
164:	learn: 0.0680685	total: 3m 16s	remaining: 6m 37s
165:	learn: 0.0680672	total: 3m 17s	remaining: 6m 36s
166:	learn: 0.0680652	total: 3m 18s	remaining: 6m 35s
167:	learn: 0.0680641	total: 3m 19s	remaining: 6m 33s
168:	learn: 0.0680625	total: 3m 20s	remaining: 6m 32s
169:	learn: 0.0680610	total: 3m 21s	remaining: 6m 31s
170:	learn: 0.0680597	total: 3m 22s	remaining: 6m 29s
171:	learn: 0.0680584	total: 3m 23s	remaining: 6m 28s
172:	learn: 0.0680555	total: 3m 24s	remaining: 6m 27s
173:	learn: 0.0680547	total: 3m 25s	remaining: 6m 25s
174:	learn: 0.0680519	total: 3m 

309:	learn: 0.0678972	total: 6m 6s	remaining: 3m 44s
310:	learn: 0.0678963	total: 6m 8s	remaining: 3m 43s
311:	learn: 0.0678950	total: 6m 9s	remaining: 3m 42s
312:	learn: 0.0678936	total: 6m 10s	remaining: 3m 41s
313:	learn: 0.0678925	total: 6m 12s	remaining: 3m 40s
314:	learn: 0.0678916	total: 6m 13s	remaining: 3m 39s
315:	learn: 0.0678900	total: 6m 14s	remaining: 3m 38s
316:	learn: 0.0678889	total: 6m 16s	remaining: 3m 37s
317:	learn: 0.0678880	total: 6m 17s	remaining: 3m 35s
318:	learn: 0.0678865	total: 6m 18s	remaining: 3m 34s
319:	learn: 0.0678855	total: 6m 20s	remaining: 3m 33s
320:	learn: 0.0678829	total: 6m 21s	remaining: 3m 32s
321:	learn: 0.0678817	total: 6m 22s	remaining: 3m 31s
322:	learn: 0.0678809	total: 6m 23s	remaining: 3m 30s
323:	learn: 0.0678801	total: 6m 24s	remaining: 3m 29s
324:	learn: 0.0678795	total: 6m 26s	remaining: 3m 27s
325:	learn: 0.0678781	total: 6m 27s	remaining: 3m 26s
326:	learn: 0.0678774	total: 6m 28s	remaining: 3m 25s
327:	learn: 0.0678766	total: 6m

463:	learn: 0.0677193	total: 9m 10s	remaining: 42.7s
464:	learn: 0.0677179	total: 9m 11s	remaining: 41.5s
465:	learn: 0.0677168	total: 9m 12s	remaining: 40.3s
466:	learn: 0.0677156	total: 9m 13s	remaining: 39.1s
467:	learn: 0.0677143	total: 9m 15s	remaining: 38s
468:	learn: 0.0677134	total: 9m 16s	remaining: 36.8s
469:	learn: 0.0677122	total: 9m 17s	remaining: 35.6s
470:	learn: 0.0677106	total: 9m 18s	remaining: 34.4s
471:	learn: 0.0677098	total: 9m 20s	remaining: 33.2s
472:	learn: 0.0677078	total: 9m 21s	remaining: 32s
473:	learn: 0.0677067	total: 9m 22s	remaining: 30.8s
474:	learn: 0.0677051	total: 9m 23s	remaining: 29.7s
475:	learn: 0.0677039	total: 9m 24s	remaining: 28.5s
476:	learn: 0.0677028	total: 9m 26s	remaining: 27.3s
477:	learn: 0.0677023	total: 9m 27s	remaining: 26.1s
478:	learn: 0.0677010	total: 9m 28s	remaining: 24.9s
479:	learn: 0.0676999	total: 9m 29s	remaining: 23.7s
480:	learn: 0.0676980	total: 9m 30s	remaining: 22.5s
481:	learn: 0.0676968	total: 9m 32s	remaining: 21.

In [75]:
for i in datelist:
    sample_submission[i] = y_pred

In [76]:
sample_submission.to_csv("submission_no_ensemble.csv", index=False)

## 3. With Ensemble (1000 iterations)

In [46]:
num_ensembles = 5
y_pred = 0.0
for i in tqdm(range(num_ensembles)):
    model = CatBoostRegressor(
        iterations=1000, 
        learning_rate=0.03,
        depth=6, 
        l2_leaf_reg=3,
        loss_function='MAE',
        eval_metric='MAE',
        cat_features = categorical_indices,
        random_seed=i)
    
    model.fit(X_train, y_train)

    y_pred += model.predict(X_test_oct_2016)
    
y_pred /= num_ensembles

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

0:	learn: 0.0688179	total: 423ms	remaining: 7m 3s
1:	learn: 0.0688018	total: 926ms	remaining: 7m 42s
2:	learn: 0.0687839	total: 1.31s	remaining: 7m 16s
3:	learn: 0.0687673	total: 1.77s	remaining: 7m 19s
4:	learn: 0.0687525	total: 2.24s	remaining: 7m 26s
5:	learn: 0.0687385	total: 2.73s	remaining: 7m 32s
6:	learn: 0.0687257	total: 3.15s	remaining: 7m 27s
7:	learn: 0.0687122	total: 3.6s	remaining: 7m 26s
8:	learn: 0.0687002	total: 4.02s	remaining: 7m 22s
9:	learn: 0.0686879	total: 4.49s	remaining: 7m 24s
10:	learn: 0.0686773	total: 4.98s	remaining: 7m 27s
11:	learn: 0.0686675	total: 5.45s	remaining: 7m 29s
12:	learn: 0.0686578	total: 5.94s	remaining: 7m 30s
13:	learn: 0.0686459	total: 6.35s	remaining: 7m 27s
14:	learn: 0.0686357	total: 6.79s	remaining: 7m 26s
15:	learn: 0.0686227	total: 7.21s	remaining: 7m 23s
16:	learn: 0.0686119	total: 7.67s	remaining: 7m 23s
17:	learn: 0.0686008	total: 8.11s	remaining: 7m 22s
18:	learn: 0.0685899	total: 8.54s	remaining: 7m 20s
19:	learn: 0.0685804	tot

159:	learn: 0.0680885	total: 1m 19s	remaining: 6m 56s
160:	learn: 0.0680869	total: 1m 19s	remaining: 6m 55s
161:	learn: 0.0680853	total: 1m 20s	remaining: 6m 55s
162:	learn: 0.0680832	total: 1m 20s	remaining: 6m 55s
163:	learn: 0.0680819	total: 1m 21s	remaining: 6m 55s
164:	learn: 0.0680804	total: 1m 22s	remaining: 6m 55s
165:	learn: 0.0680792	total: 1m 22s	remaining: 6m 54s
166:	learn: 0.0680785	total: 1m 23s	remaining: 6m 55s
167:	learn: 0.0680774	total: 1m 23s	remaining: 6m 55s
168:	learn: 0.0680762	total: 1m 24s	remaining: 6m 55s
169:	learn: 0.0680743	total: 1m 25s	remaining: 6m 55s
170:	learn: 0.0680730	total: 1m 25s	remaining: 6m 55s
171:	learn: 0.0680712	total: 1m 26s	remaining: 6m 54s
172:	learn: 0.0680692	total: 1m 26s	remaining: 6m 54s
173:	learn: 0.0680674	total: 1m 27s	remaining: 6m 54s
174:	learn: 0.0680650	total: 1m 27s	remaining: 6m 53s
175:	learn: 0.0680642	total: 1m 28s	remaining: 6m 53s
176:	learn: 0.0680633	total: 1m 28s	remaining: 6m 52s
177:	learn: 0.0680618	total:

312:	learn: 0.0678972	total: 2m 38s	remaining: 5m 47s
313:	learn: 0.0678961	total: 2m 38s	remaining: 5m 47s
314:	learn: 0.0678950	total: 2m 39s	remaining: 5m 46s
315:	learn: 0.0678938	total: 2m 39s	remaining: 5m 45s
316:	learn: 0.0678929	total: 2m 40s	remaining: 5m 45s
317:	learn: 0.0678910	total: 2m 40s	remaining: 5m 45s
318:	learn: 0.0678902	total: 2m 41s	remaining: 5m 44s
319:	learn: 0.0678893	total: 2m 42s	remaining: 5m 44s
320:	learn: 0.0678886	total: 2m 42s	remaining: 5m 43s
321:	learn: 0.0678870	total: 2m 43s	remaining: 5m 43s
322:	learn: 0.0678860	total: 2m 43s	remaining: 5m 42s
323:	learn: 0.0678848	total: 2m 44s	remaining: 5m 42s
324:	learn: 0.0678841	total: 2m 44s	remaining: 5m 42s
325:	learn: 0.0678833	total: 2m 45s	remaining: 5m 41s
326:	learn: 0.0678825	total: 2m 45s	remaining: 5m 41s
327:	learn: 0.0678822	total: 2m 46s	remaining: 5m 40s
328:	learn: 0.0678815	total: 2m 46s	remaining: 5m 39s
329:	learn: 0.0678811	total: 2m 47s	remaining: 5m 39s
330:	learn: 0.0678796	total:

465:	learn: 0.0677159	total: 3m 57s	remaining: 4m 31s
466:	learn: 0.0677131	total: 3m 57s	remaining: 4m 31s
467:	learn: 0.0677129	total: 3m 58s	remaining: 4m 31s
468:	learn: 0.0677110	total: 3m 58s	remaining: 4m 30s
469:	learn: 0.0677095	total: 3m 59s	remaining: 4m 30s
470:	learn: 0.0677078	total: 3m 59s	remaining: 4m 29s
471:	learn: 0.0677074	total: 4m	remaining: 4m 28s
472:	learn: 0.0677060	total: 4m	remaining: 4m 28s
473:	learn: 0.0677046	total: 4m 1s	remaining: 4m 27s
474:	learn: 0.0677028	total: 4m 2s	remaining: 4m 27s
475:	learn: 0.0677014	total: 4m 2s	remaining: 4m 27s
476:	learn: 0.0677001	total: 4m 3s	remaining: 4m 26s
477:	learn: 0.0676990	total: 4m 3s	remaining: 4m 25s
478:	learn: 0.0676968	total: 4m 4s	remaining: 4m 25s
479:	learn: 0.0676954	total: 4m 4s	remaining: 4m 25s
480:	learn: 0.0676942	total: 4m 5s	remaining: 4m 24s
481:	learn: 0.0676931	total: 4m 5s	remaining: 4m 23s
482:	learn: 0.0676901	total: 4m 6s	remaining: 4m 23s
483:	learn: 0.0676890	total: 4m 6s	remaining: 

...

464:	learn: 0.0677343	total: 3m 49s	remaining: 4m 23s
465:	learn: 0.0677332	total: 3m 49s	remaining: 4m 23s
466:	learn: 0.0677323	total: 3m 50s	remaining: 4m 22s
467:	learn: 0.0677306	total: 3m 50s	remaining: 4m 22s
468:	learn: 0.0677290	total: 3m 51s	remaining: 4m 21s
469:	learn: 0.0677280	total: 3m 51s	remaining: 4m 21s
470:	learn: 0.0677268	total: 3m 52s	remaining: 4m 20s
471:	learn: 0.0677246	total: 3m 52s	remaining: 4m 20s
472:	learn: 0.0677241	total: 3m 53s	remaining: 4m 19s
473:	learn: 0.0677230	total: 3m 53s	remaining: 4m 19s
474:	learn: 0.0677221	total: 3m 54s	remaining: 4m 18s
475:	learn: 0.0677200	total: 3m 54s	remaining: 4m 18s
476:	learn: 0.0677185	total: 3m 54s	remaining: 4m 17s
477:	learn: 0.0677160	total: 3m 55s	remaining: 4m 17s
478:	learn: 0.0677155	total: 3m 55s	remaining: 4m 16s
479:	learn: 0.0677130	total: 3m 56s	remaining: 4m 16s
480:	learn: 0.0677113	total: 3m 56s	remaining: 4m 15s
481:	learn: 0.0677098	total: 3m 57s	remaining: 4m 15s
482:	learn: 0.0677080	total:

618:	learn: 0.0675653	total: 5m 4s	remaining: 3m 7s
619:	learn: 0.0675645	total: 5m 4s	remaining: 3m 6s
620:	learn: 0.0675629	total: 5m 5s	remaining: 3m 6s
621:	learn: 0.0675617	total: 5m 5s	remaining: 3m 5s
622:	learn: 0.0675612	total: 5m 6s	remaining: 3m 5s
623:	learn: 0.0675602	total: 5m 6s	remaining: 3m 4s
624:	learn: 0.0675591	total: 5m 7s	remaining: 3m 4s
625:	learn: 0.0675579	total: 5m 7s	remaining: 3m 3s
626:	learn: 0.0675561	total: 5m 8s	remaining: 3m 3s
627:	learn: 0.0675548	total: 5m 8s	remaining: 3m 2s
628:	learn: 0.0675537	total: 5m 9s	remaining: 3m 2s
629:	learn: 0.0675526	total: 5m 9s	remaining: 3m 1s
630:	learn: 0.0675507	total: 5m 10s	remaining: 3m 1s
631:	learn: 0.0675498	total: 5m 10s	remaining: 3m
632:	learn: 0.0675490	total: 5m 11s	remaining: 3m
633:	learn: 0.0675479	total: 5m 11s	remaining: 2m 59s
634:	learn: 0.0675471	total: 5m 12s	remaining: 2m 59s
635:	learn: 0.0675465	total: 5m 12s	remaining: 2m 58s
636:	learn: 0.0675452	total: 5m 13s	remaining: 2m 58s
637:	le

772:	learn: 0.0674360	total: 6m 17s	remaining: 1m 50s
773:	learn: 0.0674355	total: 6m 17s	remaining: 1m 50s
774:	learn: 0.0674341	total: 6m 18s	remaining: 1m 49s
775:	learn: 0.0674325	total: 6m 18s	remaining: 1m 49s
776:	learn: 0.0674319	total: 6m 19s	remaining: 1m 48s
777:	learn: 0.0674316	total: 6m 19s	remaining: 1m 48s
778:	learn: 0.0674313	total: 6m 20s	remaining: 1m 47s
779:	learn: 0.0674302	total: 6m 20s	remaining: 1m 47s
780:	learn: 0.0674295	total: 6m 21s	remaining: 1m 46s
781:	learn: 0.0674289	total: 6m 21s	remaining: 1m 46s
782:	learn: 0.0674283	total: 6m 22s	remaining: 1m 45s
783:	learn: 0.0674273	total: 6m 22s	remaining: 1m 45s
784:	learn: 0.0674267	total: 6m 23s	remaining: 1m 44s
785:	learn: 0.0674263	total: 6m 23s	remaining: 1m 44s
786:	learn: 0.0674255	total: 6m 24s	remaining: 1m 43s
787:	learn: 0.0674246	total: 6m 24s	remaining: 1m 43s
788:	learn: 0.0674241	total: 6m 24s	remaining: 1m 42s
789:	learn: 0.0674229	total: 6m 25s	remaining: 1m 42s
790:	learn: 0.0674222	total:

926:	learn: 0.0673194	total: 7m 36s	remaining: 35.9s
927:	learn: 0.0673184	total: 7m 36s	remaining: 35.4s
928:	learn: 0.0673174	total: 7m 37s	remaining: 34.9s
929:	learn: 0.0673162	total: 7m 37s	remaining: 34.4s
930:	learn: 0.0673159	total: 7m 38s	remaining: 33.9s
931:	learn: 0.0673156	total: 7m 38s	remaining: 33.4s
932:	learn: 0.0673153	total: 7m 38s	remaining: 33s
933:	learn: 0.0673148	total: 7m 39s	remaining: 32.5s
934:	learn: 0.0673145	total: 7m 39s	remaining: 32s
935:	learn: 0.0673138	total: 7m 40s	remaining: 31.5s
936:	learn: 0.0673132	total: 7m 40s	remaining: 31s
937:	learn: 0.0673117	total: 7m 41s	remaining: 30.5s
938:	learn: 0.0673113	total: 7m 41s	remaining: 30s
939:	learn: 0.0673107	total: 7m 42s	remaining: 29.5s
940:	learn: 0.0673105	total: 7m 42s	remaining: 29s
941:	learn: 0.0673102	total: 7m 43s	remaining: 28.5s
942:	learn: 0.0673099	total: 7m 43s	remaining: 28s
943:	learn: 0.0673095	total: 7m 44s	remaining: 27.5s
944:	learn: 0.0673090	total: 7m 44s	remaining: 27s
945:	le

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [46:49<00:00, 561.80s/it]


In [47]:
for i in datelist:
    sample_submission[i] = y_pred

In [48]:
sample_submission.to_csv("submission_ensemble_1000iterations.csv", index=False)

## 4. Without Ensemble (1000 iterations)

In [43]:
model = CatBoostRegressor(
    iterations=500, 
    learning_rate=0.03,
    depth=6, 
    l2_leaf_reg=3,
    loss_function='MAE',
    eval_metric='MAE',
    cat_features = categorical_indices
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test_oct_2016)

0:	learn: 0.0688179	total: 476ms	remaining: 3m 57s
1:	learn: 0.0688018	total: 847ms	remaining: 3m 30s
2:	learn: 0.0687839	total: 1.24s	remaining: 3m 25s
3:	learn: 0.0687673	total: 1.64s	remaining: 3m 23s
4:	learn: 0.0687525	total: 2.11s	remaining: 3m 29s
5:	learn: 0.0687385	total: 2.53s	remaining: 3m 28s
6:	learn: 0.0687257	total: 2.94s	remaining: 3m 27s
7:	learn: 0.0687122	total: 3.42s	remaining: 3m 30s
8:	learn: 0.0687002	total: 3.81s	remaining: 3m 28s
9:	learn: 0.0686879	total: 4.3s	remaining: 3m 30s
10:	learn: 0.0686773	total: 4.74s	remaining: 3m 30s
11:	learn: 0.0686675	total: 5.18s	remaining: 3m 30s
12:	learn: 0.0686578	total: 5.6s	remaining: 3m 29s
13:	learn: 0.0686459	total: 6s	remaining: 3m 28s
14:	learn: 0.0686357	total: 6.42s	remaining: 3m 27s
15:	learn: 0.0686227	total: 6.8s	remaining: 3m 25s
16:	learn: 0.0686119	total: 7.23s	remaining: 3m 25s
17:	learn: 0.0686008	total: 7.68s	remaining: 3m 25s
18:	learn: 0.0685899	total: 8.1s	remaining: 3m 25s
19:	learn: 0.0685804	total: 8

158:	learn: 0.0680898	total: 1m 11s	remaining: 2m 33s
159:	learn: 0.0680885	total: 1m 12s	remaining: 2m 33s
160:	learn: 0.0680869	total: 1m 12s	remaining: 2m 32s
161:	learn: 0.0680853	total: 1m 12s	remaining: 2m 32s
162:	learn: 0.0680832	total: 1m 13s	remaining: 2m 32s
163:	learn: 0.0680819	total: 1m 13s	remaining: 2m 31s
164:	learn: 0.0680804	total: 1m 14s	remaining: 2m 31s
165:	learn: 0.0680792	total: 1m 14s	remaining: 2m 30s
166:	learn: 0.0680785	total: 1m 15s	remaining: 2m 30s
167:	learn: 0.0680774	total: 1m 16s	remaining: 2m 30s
168:	learn: 0.0680762	total: 1m 16s	remaining: 2m 29s
169:	learn: 0.0680743	total: 1m 17s	remaining: 2m 29s
170:	learn: 0.0680730	total: 1m 17s	remaining: 2m 29s
171:	learn: 0.0680712	total: 1m 18s	remaining: 2m 28s
172:	learn: 0.0680692	total: 1m 18s	remaining: 2m 28s
173:	learn: 0.0680674	total: 1m 19s	remaining: 2m 28s
174:	learn: 0.0680650	total: 1m 19s	remaining: 2m 27s
175:	learn: 0.0680642	total: 1m 20s	remaining: 2m 27s
176:	learn: 0.0680633	total:

311:	learn: 0.0678987	total: 2m 27s	remaining: 1m 28s
312:	learn: 0.0678972	total: 2m 27s	remaining: 1m 28s
313:	learn: 0.0678961	total: 2m 28s	remaining: 1m 27s
314:	learn: 0.0678950	total: 2m 28s	remaining: 1m 27s
315:	learn: 0.0678938	total: 2m 29s	remaining: 1m 26s
316:	learn: 0.0678929	total: 2m 29s	remaining: 1m 26s
317:	learn: 0.0678910	total: 2m 30s	remaining: 1m 26s
318:	learn: 0.0678902	total: 2m 30s	remaining: 1m 25s
319:	learn: 0.0678893	total: 2m 31s	remaining: 1m 25s
320:	learn: 0.0678886	total: 2m 31s	remaining: 1m 24s
321:	learn: 0.0678870	total: 2m 32s	remaining: 1m 24s
322:	learn: 0.0678860	total: 2m 32s	remaining: 1m 23s
323:	learn: 0.0678848	total: 2m 33s	remaining: 1m 23s
324:	learn: 0.0678841	total: 2m 34s	remaining: 1m 23s
325:	learn: 0.0678833	total: 2m 34s	remaining: 1m 22s
326:	learn: 0.0678825	total: 2m 35s	remaining: 1m 22s
327:	learn: 0.0678822	total: 2m 35s	remaining: 1m 21s
328:	learn: 0.0678815	total: 2m 36s	remaining: 1m 21s
329:	learn: 0.0678811	total:

466:	learn: 0.0677131	total: 3m 40s	remaining: 15.6s
467:	learn: 0.0677129	total: 3m 41s	remaining: 15.1s
468:	learn: 0.0677110	total: 3m 41s	remaining: 14.6s
469:	learn: 0.0677095	total: 3m 42s	remaining: 14.2s
470:	learn: 0.0677078	total: 3m 42s	remaining: 13.7s
471:	learn: 0.0677074	total: 3m 42s	remaining: 13.2s
472:	learn: 0.0677060	total: 3m 43s	remaining: 12.8s
473:	learn: 0.0677046	total: 3m 43s	remaining: 12.3s
474:	learn: 0.0677028	total: 3m 44s	remaining: 11.8s
475:	learn: 0.0677014	total: 3m 44s	remaining: 11.3s
476:	learn: 0.0677001	total: 3m 45s	remaining: 10.9s
477:	learn: 0.0676990	total: 3m 45s	remaining: 10.4s
478:	learn: 0.0676968	total: 3m 46s	remaining: 9.92s
479:	learn: 0.0676954	total: 3m 46s	remaining: 9.44s
480:	learn: 0.0676942	total: 3m 47s	remaining: 8.97s
481:	learn: 0.0676931	total: 3m 47s	remaining: 8.5s
482:	learn: 0.0676901	total: 3m 47s	remaining: 8.02s
483:	learn: 0.0676890	total: 3m 48s	remaining: 7.55s
484:	learn: 0.0676882	total: 3m 48s	remaining: 

In [44]:
for i in datelist:
    sample_submission[i] = y_pred

In [45]:
sample_submission.to_csv("submission_no_ensemble_1000iterations.csv", index=False)